In [4]:
channels = ["WW","bb","tautau","mumu","nuenue","numunumu","nutaunutau"]
# channels = ["bb"]

profile = ['NFW', 'Burkert']
masses = {"WW":[90, 8000], "bb":[10, 8000], 'tautau':[5, 4000], 'mumu':[5, 1000], "nuenue":[5, 200],"numunumu":[5, 200],"nutaunutau":[5,200]}

In [5]:

cfig_file = '/home/tchau/code/condor_submit/UL/dagman.config'
with open(cfig_file, 'w') as c:
    c.write('DAGMAN_MAX_JOBS_SUBMITTED=500'+'\n')
    c.write('DAGMAN_MAX_SUBMIT_PER_INTERVAL=20'+'\n')
    c.write('DAGMAN_USER_LOG_SCAN_INTERVAL=10'+'\n')
    c.close()

sub_file = '/home/tchau/code/condor_submit/UL/UL.submit'

with open(sub_file, 'w') as s:
    s.write('executable = /data/user/tchau/Sandbox/GC_OscNext/Sensitivity/UpperLimit_LLH-Interval.py'+'\n')
    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write('logdir = /scratch/tchau/DM_OscNext/MakePDF/logs/'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 2GB'+'\n')
    s.write('\n')
    s.write('arguments = -c $(c) -p $(p) -u $(u) -l $(l) -n $(n)'+'\n')

    s.write('queue')
    s.close()    

In [6]:
import os
submit = '/home/tchau/code/condor_submit/UL/submit.sh'
n = 10
with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    for c in channels:
        for p in profile:
            path = '/home/tchau/code/condor_submit/UL/{}_{}_{}'.format(c, p, n)
            if not (os.path.exists(path)): os.makedirs(path)
            dag_file = '{}/{}_{}_{}.dag'.format(path, c, p, n)
            with open(dag_file, 'w') as f:
                f.write('JOB UL_{}_{}_{}.dag'.format(c,p, n)+' UL.submit \n')
                f.write('VARS UL_{}_{}_{}.dag'.format(c,p, n)+' JOBNAME="UL_{}_{}_{}"'.format(c, p, n) +' c="{}" p="{}" u="{}" l="{}" n="{}"'.format(c, p, masses[c][1], masses[c][0], n)+'\n')
            f.close()
            s.write("condor_submit_dag -config dagman.config {}\n".format(dag_file))
            print("condor_submit_dag -config dagman.config {}\n".format(dag_file))
s.close()

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/WW_NFW_10/WW_NFW_10.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/WW_Burkert_10/WW_Burkert_10.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/bb_NFW_10/bb_NFW_10.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/bb_Burkert_10/bb_Burkert_10.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/tautau_NFW_10/tautau_NFW_10.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/tautau_Burkert_10/tautau_Burkert_10.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/mumu_NFW_10/mumu_NFW_10.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/mumu_Burkert_10/mumu_Burkert_10.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/nuenue_NFW_10/nuenue_NFW_10.dag

condor_submit_dag -config dagman.config /home